#### 1. Creating Initial Variables

In [ ]:
import sagemaker
from sagemaker.tensorflow import TensorFlow
import boto3
import os

# SageMaker session and role
role = sagemaker.get_execution_role()
sess = sagemaker.Session()
bucket = sess.default_bucket()

# S3 paths
s3_prefix = "cnn-script-mode-workflow"
cnn_s3_prefix = f"{s3_prefix}/cnn"
cnn_train_s3_uri = f"s3://{bucket}/{cnn_s3_prefix}/train"

# Directory containing .cnn files
train_dir = os.path.join(os.getcwd(), "")

#### 2. Uploading Training Data to S3

In [ ]:
s3_resource_bucket = boto3.Session().resource("s3").Bucket(bucket)
s3_resource_bucket.Object(os.path.join(cnn_s3_prefix, "train.cnn")).upload_file(
    train_dir + "/train.cnn"
)

#### 3. Creating Estimator (w/ Hyperparameters and Paramaters)

In [ ]:
# Hyperparameters
hyperparameters = {
    "epochs": 3,
    "batch_size": 32
}

entry_point = "image-recognition-script.py"
output_path = f"s3://{bucket}/{s3_prefix}/output"
train_instance_type = "ml.g4dn.4xlarge" # GPU instance

# Estimator parameters
estimator_parameters = {
    "entry_point": entry_point,
    "source_dir": "scripts",
    "framework_version": "2.4",
    "py_version": "py37",
    "instance_type": train_instance_type,
    "instance_count": 1,
    "hyperparameters": hyperparameters,
    "role": role,
    "base_job_name": "cnn-model",
    "output_path": output_path,
}

estimator = TensorFlow(**estimator_parameters)

#### 4. Training

In [ ]:
inputs = {
    "train": cnn_train_s3_uri
}

estimator.fit(inputs)

#### 5. Deploying the Model to an Endpoint

In [ ]:
cnn_predictor = estimator.deploy(
    initial_instance_count=1,
    instance_type="ml.g4dn.4xlarge", # GPU instance
    endpoint_name="cnn-endpoint-2"
)

#### 6. Predictions

In [ ]:
import pickle
import numpy as np

# Prepare test data
def getTestData():
    def unpickle(file):
        with open(file, 'rb') as fo:
            dict = pickle.load(fo, encoding='bytes')
        return dict
    fileNameTestBatch = './cifar-10-batches-py/test_batch'
    test_data = unpickle(fileNameTestBatch)[b'data']
    test_data = test_data.reshape(-1, 32, 32, 3) / 255.0
    test_labels = np.array(unpickle(fileNameTestBatch)[b'labels'])
    
    num_samples_to_select = 600
    random_indices = np.random.choice(test_data.shape[0], num_samples_to_select, replace=False)
    selected_test_data = test_data[random_indices]
    selected_test_labels = test_labels[random_indices]
    
    return selected_test_data, selected_test_labels

test_data, test_labels = getTestData()

from sklearn.metrics import accuracy_score
def getAccuracyOfPrediction(cnn_predictions, test_labels):
    cnn_predicted_labels = np.argmax(cnn_predictions, axis=1)
    accuracy = accuracy_score(test_labels, cnn_predicted_labels)
    print("Accuracy:", accuracy)

#### 7. Deleting the Endpoint

In [ ]:
sagemaker.Session().delete_endpoint("cnn-endpoint-2")
sagemaker.Session().delete_endpoint_config("cnn-endpoint-2")